In [106]:
import pandas as pd
import numpy as np
import my_pickle as mp
import my_functions as mf
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.ensemble import RandomForestRegressor

import my_resample as ms

import warnings
warnings.filterwarnings('ignore')


from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble.partial_dependence import plot_partial_dependence


from importlib import reload

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from random import *
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score

import my_pickle as mp
import my_features as mf
import my_resample as ms
# from my_resample import div_count_pos_neg, undersample, oversample
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
from scipy import interp


from sklearn.linear_model import LogisticRegression


from sklearn.ensemble.partial_dependence import plot_partial_dependence
from sklearn.ensemble.partial_dependence import partial_dependence

In [20]:
# Load User Data
user_df = mp.unjson_it('data_user')
user_df = mp.reinstate_date(user_df,['activeAt', 'available', 'birthday','created','updated'])

# Load Conversation Data
convo_df = mp.unjson_it('data_convo')

In [ ]:
user_df.rename('uid')
# user_df.reset_index()

In [21]:
uid_sent_message = convo_df.uid_sender.unique()

In [48]:
# create dictionaries of mesages sent and responses received
convo_df['sent'] = 1
messages_sent_by_uid = pd.DataFrame(convo_df.groupby('uid_sender').sent.count())
responses_received_by_uid = convo_df.groupby('uid_sender').response.sum()

In [52]:
big = pd.concat([user_df, messages_sent_by_uid], axis=1)
big = pd.concat([big, responses_received_by_uid], axis=1)
big['ratio'] = big.response/big.sent

In [55]:
big = big[big.sent > 0]

In [56]:
big.head()

,I_count,I_ratio,about,activeAt,age,amenities,available,birthday,college,created,...,sentence_ratio,smokingOk,term,timeframe,type,updated,work,sent,response,ratio
013LzOrVju,2.0,0.006803,Integrative Medicine practitioner. Pain Manage...,2017-10-09 06:43:43.816000000,30.0,"[A/C, Furnished]",2017-10-04 19:54:47,1988-01-01,None,2017-10-01 02:52:10.131,...,0.030612,1.0,4.0,3.205569e+08,NaN,2018-01-12 19:41:02.853,Entrepreneur,10.0,1.0,0.100000
01iDVeaXl1,0.0,NaN,,2017-09-06 23:34:22.711000064,25.0,"[A/C, Dishwasher, Laundry, Parking, Private Bath]",2017-09-10 12:50:51,1993-01-01,None,2017-09-06 19:50:02.050,...,NaN,1.0,12.0,3.204490e+08,1.0,2018-01-12 17:23:52.739,Icell,4.0,0.0,0.000000
02ICQW0SJo,1.0,0.008000,Hi guys looking for a female roommate with com...,2018-01-04 18:47:38.100000000,35.0,None,NaT,1983-01-01,Southwestern College,2017-09-02 16:37:51.414,...,0.024000,0.0,NaN,NaN,NaN,2018-01-12 16:53:53.167,UnionBank,7.0,4.0,0.571429
02Wvx6BcIx,2.0,0.016393,"Hello! My name is Sing, I'm 30, married and lo...",2017-09-14 23:37:18.512999936,32.0,"[A/C, Laundry, Parking, Private Bath]",2017-10-01 08:39:15,1986-01-01,University of Southern California,2017-09-14 15:38:14.056,...,0.016393,0.0,6.0,1.443661e+09,1.0,2018-01-12 18:17:26.328,None,1.0,0.0,0.000000
02omVxM8Xm,0.0,0.000000,The perfect place... wya ?,2017-09-23 05:57:10.188999936,22.0,None,NaT,1996-01-01,None,2017-09-11 02:33:39.710,...,0.148148,1.0,NaN,NaN,NaN,2018-01-12 17:53:16.833,Delhi Palace,1.0,0.0,0.000000


In [100]:
def get_len(n):
    try:
        return len(n)
    except:
        return 0
    
get_length = ['amenities','hobbies', 'neighborhoods']

keep_X = ['I_count', 'I_ratio',  'age', 'exclaim_count',
       'exclaim_ratio','gender', 'has_about',
       'has_facebookId', 'has_linkedinId', 'has_picture', 'has_room',
                 'inRelationship', 'isClean', 'isNight', 'isStudent', 'len_about',
              'maxCost',  'minCost', 'numRoommates', 
            'period_count',
       'period_ratio', 'petsOk',
               'question_count',
       'question_ratio', 'sentence_count', 'sentence_ratio', 'smokingOk',
       'term', 'type']
keep_y = ['ratio']
drop = ['activeAt','available', 'birthday','college', 'created',   'facebookId',     
          'hometownCity', 'hometownCountry', 'hometownState','about', 
             'linkedinId', 'location', 'onboarded',   'picture', 'sent', 'response',
          'updated', 'work','timeframe', 'metro',]

for col in get_length:
    big[col] = big[col].apply(get_len)

X = big[keep_X]
y = big[keep_y]

In [101]:
X.iloc[:,10:20].head()

,has_room,inRelationship,isClean,isNight,isStudent,len_about,maxCost,minCost,numRoommates,period_count
013LzOrVju,1.0,0.0,0.0,0.0,0.0,294.0,2000.0,400.0,1.0,8.0
01iDVeaXl1,0.0,1.0,1.0,1.0,0.0,0.0,500.0,300.0,3.0,0.0
02ICQW0SJo,1.0,0.0,1.0,1.0,1.0,125.0,NaN,NaN,NaN,3.0
02Wvx6BcIx,0.0,1.0,1.0,0.0,0.0,122.0,800.0,300.0,1.0,1.0
02omVxM8Xm,1.0,0.0,1.0,1.0,0.0,27.0,NaN,NaN,NaN,3.0


In [102]:
X = X.fillna(X.mean())

In [96]:
X.head()
X.columns

Index(['I_count', 'I_ratio', 'age', 'exclaim_count', 'exclaim_ratio', 'gender',
       'has_about', 'has_facebookId', 'has_linkedinId', 'has_picture',
       'has_room', 'inRelationship', 'isClean', 'isNight', 'isStudent',
       'len_about', 'maxCost', 'metro', 'minCost', 'numRoommates',
       'period_count', 'period_ratio', 'petsOk', 'question_count',
       'question_ratio', 'sentence_count', 'sentence_ratio', 'smokingOk',
       'term', 'type'],
      dtype='object')

In [77]:
print((y.ratio >= .1).sum())
print((y.ratio < .1).sum())

652
1869


In [103]:
y['binary'] = y.ratio.apply(lambda x: int(x+.9))

In [105]:
def display_importances_trees():
    # show feature importances
    feature_df = pd.DataFrame([X.columns, model.feature_importances_]).T
    feature_df.columns = ['feature','value']
    return feature_df.sort_values('value', ascending=False)
             
def display_metrics():
    print("\nMETRICS")
    print("Model recall: {}".format(recall_score(y_test, y_pred)))
    print("Model precision: {}".format(precision_score(y_test, y_pred)))
    print("Model accuracy: {}".format(model.score(X_test, y_test)))

    print ("\nCONFUSION MATRIX")
    print (confusion_matrix(y_test, y_pred))
    print ("\nkey:")
    print (" TN   FP ")
    print (" FN   TP ")

    print("\nRECALL AND ACCURACY FOR DIFFERNET MODELS")
    print("recall     \t precision   \tmodel")
    print(recall_score(y_test, y_pred), '\t',precision_score(y_test, y_pred), "my model")
    print(recall_score(y_test, pred_all_0),'\t','\t', precision_score(y_test, pred_all_0), "\t\tpredict all zero")
    print(recall_score(y_test, pred_all_1),'\t','\t', precision_score(y_test, pred_all_1), "predict all one")
    print(recall_score(y_test, pred_50_50),'\t', precision_score(y_test, pred_50_50), "predict 50-50")
    print(recall_score(y_test, pred_90_10), precision_score(y_test, pred_90_10), "predict 90-10")

In [118]:
y.head()

,ratio,binary
013LzOrVju,0.100000,1
01iDVeaXl1,0.000000,1
02ICQW0SJo,0.571429,1
02Wvx6BcIx,0.000000,1
02omVxM8Xm,0.000000,1


In [119]:
cutoff = .0001
y['binary'] = y.ratio.apply(lambda x: int(x+(1-cutoff)))

In [120]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y.binary, random_state=17)

# # resample
X_train, y_train = ms.oversample(X_train, y_train, .5)

# scale data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# make fake data
pred_all_0 = [0]*len(y_test)
pred_all_1 = [1]*len(y_test)
pred_50_50 = np.random.choice([0,1], size=len(y_test))
pred_90_10 = np.random.choice([0,1], size=len(y_test), p=[.9,.1])

In [121]:
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# show metrics
display_metrics()

# show importances
display_importances_trees().head(5)


METRICS
Model recall: 0.44508670520231214
Model precision: 0.3632075471698113
Model accuracy: 0.6339144215530903

CONFUSION MATRIX
[[323 135]
 [ 96  77]]

key:
 TN   FP 
 FN   TP 

RECALL AND ACCURACY FOR DIFFERNET MODELS
recall     	 precision   	model
0.445086705202 	 0.36320754717 my model
0.0 	 	 0.0 		predict all zero
1.0 	 	 0.274167987322 predict all one
0.49710982659 	 0.267080745342 predict 50-50
0.138728323699 0.387096774194 predict 90-10


,feature,value
16,maxCost,0.11455
15,len_about,0.106094
1,I_ratio,0.0963303
17,minCost,0.0889353
2,age,0.08694


In [108]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# show metrics
display_metrics()

# show importances
display_importances_trees().head(5)



METRICS
Model recall: 0.16875
Model precision: 0.3333333333333333
Model accuracy: 0.7036450079239303

CONFUSION MATRIX
[[417  54]
 [133  27]]

key:
 TN   FP 
 FN   TP 

RECALL AND ACCURACY FOR DIFFERNET MODELS
recall     	 precision   	model
0.16875 	 0.333333333333 my model
0.0 	 	 0.0 		predict all zero
1.0 	 	 0.253565768621 predict all one
0.475 	 0.26116838488 predict 50-50
0.075 0.196721311475 predict 90-10


,feature,value
16,maxCost,0.106495
2,age,0.106101
17,minCost,0.0917003
15,len_about,0.0633065
1,I_ratio,0.0552653


In [109]:
# fit model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# show metrics
display_metrics()

# show importances
display_importances_trees().head(5)


METRICS
Model recall: 0.25625
Model precision: 0.24848484848484848
Model accuracy: 0.6148969889064976

CONFUSION MATRIX
[[347 124]
 [119  41]]

key:
 TN   FP 
 FN   TP 

RECALL AND ACCURACY FOR DIFFERNET MODELS
recall     	 precision   	model
0.25625 	 0.248484848485 my model
0.0 	 	 0.0 		predict all zero
1.0 	 	 0.253565768621 predict all one
0.475 	 0.26116838488 predict 50-50
0.075 0.196721311475 predict 90-10


,feature,value
2,age,0.110784
16,maxCost,0.10087
17,minCost,0.0960483
25,sentence_ratio,0.0664574
27,term,0.0608511


In [19]:
user_df['flag'] = True
old_names = user_df.columns
new_names = {old:old+'_sender' for old in old_names}
master_df = convo_df.join(user_df, on='uid_sender').join(user_df, on='uid_receiver', rsuffix="_receiver").rename(index=str, columns = new_names)
master_df.head(2)

NameError: name 'convo_df' is not defined

In [88]:
X.amenities = X.amenities.apply(lambda x: len(x) if isinstance(x,list) else 0)
X.hobbies = X.hobbies.apply(lambda x: len(x) if isinstance(x,list) else 0)
X.neighborhoods = X.neighborhoods.apply(lambda x: len(x) if isinstance(x,list) else 0)
X.location = X.location.apply(lambda x: 1 if isinstance(x,list) else 0)
X['len_about'] = X.about.apply(len)
X['has_about'] = X['len_about'].apply(lambda x: x>0)
X['has_about']= X['has_about'].map(TF)
X.college = X.college.apply(lambda x: 1 if isinstance(x,str) else 0)
X.hometownCity = X.hometownCity.apply(lambda x: 1 if isinstance(x,str) else 0)
X.hometownCountry = X.hometownCountry.apply(lambda x: 1 if isinstance(x,str) else 0)
X.hometownState = X.hometownState.apply(lambda x: 1 if isinstance(x,str) else 0)
X.work = X.work.apply(lambda x: 1 if isinstance(x,str) else 0)

TF = {True:1, False:0}
TF_col = ['facebookId','has_about','has_room','linkedinId','picture','has_about']
for col in TF_col:
    X[col]= X[col].map(TF)
to_drop = ['type','uid','about','metro','picture','facebookId','linkedinId']
X = X.drop(to_drop, axis=1)

AttributeError: 'DataFrame' object has no attribute 'amenities'

In [ ]:

def get_recall_preciz():
    model = GradientBoostingClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # show metrics
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    return recall_score(y_test, y_pred), precision_score(y_test, y_pred)

r = []
p = []
for w in np.arnage(.1,.9,.1):
pred_50_50 = np.random.choice([0,1], size=len(y_test))
pred_90_10 = np.random.choice([0,1], size=len(y_test), p=[.9,.1])